In [1]:
# IPython Imports
# Used to make notebook wider, comment out for normal notebook
from IPython.core.display import display, HTMLCylinders
display(HTML("<style>.container { width:90% !important; }</style>"))
from IPython.display import clear_output

ImportError: cannot import name 'HTMLCylinders' from 'IPython.core.display' (/usr/lib/python3.8/site-packages/IPython/core/display.py)

In [ ]:
# Import Libraries
import boto3
import csv
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import re
from typing import List, Dict, Callable

In [ ]:
# GLOBAL HELPER FUNCTIONS
def print_ref(d_obj:Dict[str, any], row_len:int=4) -> None:
    ref = \
f""" ---------------------------
| OBJECT KEYS FOR REFERENCE |
 ---------------------------
"""
    row = 0
    for key in d_obj.keys():
        if row == row_len: 
            row = 0
            ref += "\n"
        ref += f" {key} |"
        row += 1
    print(ref)
    
def parse_col(col_list:List[str], data_frame:pd.DataFrame=None) -> List[str]:
    res, len_ = [], len(col_list) + 1
    df_exists = data_frame is not None
    for num, col in enumerate(col_list):
        example = data_frame[col].iloc[0] if df_exists else "None"
        ans = input(f"""
Item #: {num + 1}/{len_}
Column: {col}
Example: {example}
[y/n] Default[n] -> """).lower()
        clear_output()
        if ans == "y":
            res.append(col)
    return res

def arrange_col(col_list:List[str], data_frame:pd.DataFrame=None) -> List[str]:
    cur, cols, i = [], list(col_list), 0
    df_exists = data_frame is not None
    
    while cols:
        if i == len(cols) - 1:
            i = 0
            
        x = cols[i]
        example = data_frame[x].iloc[0] if df_exists else "None"
        
        ans = input(f"""
Original:{cols}


Current: {cur}

------------------
Column: {x}

Example: {example}
""").lower()
        
        if ans == "y":
            cur.append(cols.pop(i))
        else:
            i += 1
        clear_output()
        
    
    return cur


In [ ]:
# GLOBAL VARS
BKT_NAME = 'ds-data-2020'

# DATASET NAMES
the_cc = "thecarconnectiondataset.csv" # Main Dataset
motortrend = "motortrend1974.csv" # Currently Not Using
car_feat_msrp = "carfeaturesmsrp.csv" # Currently Not Using

# SET DATA PATH/KEYS
KEY = the_cc

In [ ]:
# Create boto3 s3 Client Object
s3 = boto3.client('s3')
# Create s3 Bucket Object -> Dict
obj = s3.get_object(Bucket=BKT_NAME, Key=KEY)

# obj_keys = obj.keys()
print_ref(obj)


In [ ]:
# Convert Bucket File to transposed pd.DataFrame
auto_df = pd.read_csv(obj['Body'], index_col=0, header=None, low_memory=False).T

In [2]:
# Rename known NaN columns
auto_df.rename(columns={ auto_df.columns[0]: "Model" }, inplace = True)

NameError: name 'auto_df' is not defined

In [ ]:
# Get only wanted columns
# col = parse_col(auto_df.columns, auto_df)

# col = ["Model", 'MSRP', 'Gas Mileage', 'Engine', 'EPA Class', 'Style Name', 'Drivetrain', 'Passenger Capacity', 'Passenger Doors', 'Body Style', 'Transmission', 'Base Curb Weight (lbs)', 'Wheelbase (in)', 'Min Ground Clearance (in)', 'Track Width, Front (in)', 'Track Width, Rear (in)', 'Height, Overall (in)', 'Fuel Economy Est-Combined (MPG)', 'SAE Net Torque @ RPM', 'Fuel System', 'Engine Type', 'SAE Net Horsepower @ RPM', 'Displacement', 'First Gear Ratio (:1)', 'Sixth Gear Ratio (:1)', 'Fourth Gear Ratio (:1)', 'Seventh Gear Ratio (:1)', 'Second Gear Ratio (:1)', 'Reverse Ratio (:1)', 'Fifth Gear Ratio (:1)', 'Eighth Gear Ratio (:1)', 'Third Gear Ratio (:1)', 'Final Drive Axle Ratio (:1)', 'Steering Type', 'Front Tire Size', 'Rear Tire Size']

In [ ]:
# Save wanted col to txt file
# with open("data/wanted_col.txt", "w") as text_file:
#     for item in col:
#         text_file.write(item + "\n")

In [ ]:
# Get list of column from wanted_col.txt
wanted_col = open("data/wanted_col.txt", "r").read().splitlines()
# Extract columns to new df
auto_df_clean = auto_df[wanted_col].copy()

In [ ]:
# Clean Model Column to only contain year and name
auto_df_clean["Model"] =  [i.partition('Specs')[0] for i in auto_df_clean.Model]

In [ ]:
# Create Year Column from Model Column
auto_df_clean.insert(0, "Year", [re.search("([^\s]+)",x).group() for x in auto_df_clean["Model"]])

In [ ]:
# Cast Year Column to Int
auto_df_clean["Year"] = auto_df_clean["Year"].astype(int) 

In [ ]:
# Clean Model Column to only contain name
auto_df_clean["Model"] =  [(re.search("\s(.*)", i).group()) for i in auto_df_clean["Model"]]

In [ ]:
# Create Brand Column from Model Column
auto_df_clean.insert(1, "Brand", [re.search("([^\s]+)", x).group() for x in auto_df_clean["Model"]])

In [ ]:
# Clean Model Column to only contain model
auto_df_clean["Model"] =  [(re.search("\s(.*)", i).group(1)) for i in auto_df_clean.Model]

In [ ]:
# Get rid of all nan values in Gas Mileage
nans = pd.notnull(auto_df_clean['Gas Mileage'])
auto_df_clean = auto_df_clean[nans]

In [ ]:
# Split Gas Mileage into City and Hwy
auto_df_clean.insert(4, "Gas Mileage (City)", [(x.partition("/")[0].partition(" ")[0]) for x in auto_df_clean["Gas Mileage"]])
auto_df_clean.insert(5, "Gas Mileage (Hwy)", [(x.partition("/")[2].partition(" ")[0]) for x in auto_df_clean["Gas Mileage"]])

In [ ]:
# Gas Mileage to INT
auto_df_clean['Gas Mileage (City)'] = pd.to_numeric(auto_df_clean['Gas Mileage (City)'])
auto_df_clean['Gas Mileage (Hwy)'] = pd.to_numeric(auto_df_clean['Gas Mileage (Hwy)'])

In [ ]:
# Drop Gas Mileage Column after split
auto_df_clean.drop(columns=["Gas Mileage"], inplace=True)

In [ ]:
# Clean Gear Ratio Columns
gears = ["First Gear Ratio (:1)", "Second Gear Ratio (:1)", "Third Gear Ratio (:1)", "Fourth Gear Ratio (:1)", "Fifth Gear Ratio (:1)", "Sixth Gear Ratio (:1)", "Seventh Gear Ratio (:1)", "Eighth Gear Ratio (:1)", "Reverse Ratio (:1)", "Final Drive Axle Ratio (:1)"]

# Reorder Gears Numerically
for idx, column in enumerate(gears, 26):
    col = auto_df_clean[column]
    auto_df_clean.drop(labels=[column], axis=1,inplace = True)
    auto_df_clean.insert(idx, column, col)
    
# Cast NaN to values in col with "- TBD -"
for gear in gears:
    auto_df_clean.loc[auto_df_clean[gear].str.contains("TBD", na=False), gear] = np.nan

In [ ]:
# Cast float to values in col with string arithmetic
"Notes: Some gear columns need to be sep with (,), (/), or (" ") delimiters"
for gear in gears:
    auto_df_clean[gear] = pd.to_numeric([
        (x.partition(",")[0]) if ((type(x) == str) and ("," in x)) 
        else (x.partition("/")[0]) if ((type(x) == str) and ("/" in x))
        else (x.partition("-")[0]) if ((type(x) == str) and ("-" in x))
        else (x.partition(" ")[0]) if ((type(x) == str) and (" " in x))
        else x for x in auto_df_clean[gear]
    ], downcast="float", errors="coerce")

In [ ]:
# Cast MSRP to Numeric Float
auto_df_clean["MSRP"] = pd.to_numeric((auto_df_clean["MSRP"].replace('[\$,]', '', regex=True)), downcast="integer", errors="coerce")
# Rename MSRP to indicate $
auto_df_clean.rename(columns={ "MSRP": "MSRP($)" }, inplace = True)

In [ ]:
# Get rid of all nan values in Horsepower/Torque
nans = pd.notnull(auto_df_clean['SAE Net Torque @ RPM'])
auto_df_clean = auto_df_clean[nans]

In [ ]:
# Split SAE Horsepower into Horsepower and @ RPM
auto_df_clean.insert(4, "Horsepower", [(x.partition("@")[0]) for x in auto_df_clean["SAE Net Horsepower @ RPM"]])
auto_df_clean.insert(5, "@ RPM (HP)", [(x.partition("@")[2]) for x in auto_df_clean["SAE Net Horsepower @ RPM"]])

In [ ]:
# Split SAE Torque into Torque and @ RPM
auto_df_clean.insert(6, "Torque", [(x.partition("@")[0]) for x in auto_df_clean["SAE Net Torque @ RPM"]])
auto_df_clean.insert(7, "@ RPM (TQ)", [(x.partition("@")[2]) for x in auto_df_clean["SAE Net Torque @ RPM"]])

In [ ]:
# Clean and Cast Power Values (Horsepower / Torque)
pow_rpm_ratings = ["Horsepower", "Torque", "@ RPM (HP)", "@ RPM (TQ)"]
"Notes: Some Power columns need to be sep with (@), or (-) delimiters"
# Cast NaN to values in col with empty ratings or "- TBD -"
for col in pow_rpm_ratings:
    auto_df_clean.loc[auto_df_clean[col].str.contains("TBD", na=False), col] = np.nan
    if col in pow_rpm_ratings[:2]:
        auto_df_clean[col] = pd.to_numeric(auto_df_clean[col], downcast="float", errors="coerce")

In [ ]:
# Drop SAE Horsepower and SAE Torque Columns after splits
auto_df_clean.drop(columns=["SAE Net Horsepower @ RPM"], inplace=True)
auto_df_clean.drop(columns=["SAE Net Torque @ RPM"], inplace=True)

In [ ]:
# Clean Base Curb Weight and Cast to INT
auto_df_clean["Base Curb Weight (lbs)"] = pd.to_numeric(auto_df_clean["Base Curb Weight (lbs)"], downcast="integer", errors="coerce")

In [ ]:
# Clean Displacement and Cast to INT
auto_df_clean["Displacement"] = pd.to_numeric(
    [(x.partition("L")[0]) if type(x) == str else x for x in auto_df_clean["Displacement"]], 
    downcast="float", 
    errors="coerce"
)
auto_df_clean.rename(columns={ "Displacement": "Displacement(L)" }, inplace = True)

In [ ]:
# Clean Passenger Doors and Cast to INT
auto_df_clean["Passenger Doors"] = pd.to_numeric(auto_df_clean["Passenger Doors"], downcast="integer", errors="raise")

In [ ]:
# Clean Combined Fuel Economy Doors and Cast to INT
auto_df_clean["Fuel Economy Est-Combined (MPG)"] = pd.to_numeric(auto_df_clean["Fuel Economy Est-Combined (MPG)"], downcast="integer", errors="coerce")
auto_df_clean.rename(columns={ "Fuel Economy Est-Combined (MPG)": "Gas Mileage (Combined)" }, inplace = True)

In [ ]:
# # Save Rearranged col to txt file
# with open("data/rearranged_col.txt", "w") as text_file:
#     for item in res:
#         text_file.write(item + "\n")

# Rearrange All Columns before Export (read from rearranged_col.txt)
order = []
with open("data/rearranged_col.txt", "r") as text_file:
    order = [x.strip() for x in text_file.readlines()]
auto_df_clean = auto_df_clean[order]

In [ ]:
# Clean Engine Type

# Drop Toyota Mirai (Hydrogen Powered Car)
nans = pd.notnull(auto_df_clean['Engine Type'])
auto_df_clean = auto_df_clean[nans]

# Change Buick Regal Engine Type to "Turbo Premium Unleaded I-4"
auto_df_clean[auto_df_clean["Engine Type"] == "Turbocharged"] = "Turbo Premium Unleaded I-4"
# Change Buick Regal Engine Type to "Turbo Premium Unleaded I-4"
auto_df_clean[auto_df_clean["Engine Type"] == "Turbocharged Gas"] = "Turbo Premium Unleaded I-4"
# Change Chevrolet Cruze Engine Type to "Turbo Premium Unleaded I-4"
auto_df_clean[auto_df_clean["Engine Type"] == "Turbo Diesel"] = "Turbo Diesel 4-cylinder"


# Parse Engine Type for Cylinders and Insert Feature
auto_df_clean.insert(8, "Cylinders", auto_df_clean["Engine Type"].str.extract('(\d+)'))

In [ ]:
# Cast INT to Cylinders
auto_df_clean["Cylinders"] = pd.to_numeric(auto_df_clean["Cylinders"], downcast="integer", errors="raise")

In [ ]:
# Clean Every String Cell by Strip
auto_df_clean = auto_df_clean.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
# auto_df_clean.head()

In [ ]:
# auto_df_clean.describe()

In [ ]:
# auto_df_clean.info()

In [ ]:
# Sandbox

# sum(pd.isnull(auto_df_clean["Cylinders"]))
# auto_df_clean[pd.isnull(auto_df_clean["Cylinders"])]

# auto_df_clean[auto_df_clean["Engine Type"] == "Turbo Diesel"]
# auto_df_clean[auto_df_clean["Engine Type"] == "Turbocharged Gas"]